# Импорты

In [1]:
from transformers import AutoImageProcessor, SwinForImageClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoFeatureExtractor
import torch
import evaluate
import numpy as np
from datasets import load_dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


# Dataset

In [2]:
dataset = load_dataset("mvkvc/artifact-100k") # subset of ArtiFact labled with "ai" and "real" tags
dataset = dataset.with_format(type="torch", device=device)

batch_size = 20

# small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

model_name= 'microsoft/swin-base-patch4-window7-224'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['label'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)
C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


# Model Creation

In [3]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
  # function which calculates accuracy for a certain set of predictions
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

def collate_fn(batch):
  #data collator
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [5]:
labels = dataset['train'].features['label'].names

model = SwinForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes = True,
)

training_args = TrainingArguments(
    f"test_trainer",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)



config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

[codecarbon INFO @ 02:53:13] [setup] RAM Tracking...
[codecarbon INFO @ 02:53:13] [setup] GPU Tracking...
[codecarbon INFO @ 02:53:13] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:53:13] [setup] CPU Tracking...
[codecarbon WARNING @ 02:53:13] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 02:53:14] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-8600K CPU @ 3.60GHz
[codecarbon INFO @ 02:53:14] >>> Tracker's metadata:
[codecarbon INFO @ 02:53:14]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 02:53:14]   Python version: 3.9.13
[codecarbon INFO @ 02:53:14]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 02:53:14]   Available RAM : 15.866 GB
[codecarbon INFO @ 02:53:14]   CPU count: 6
[codecarbon INFO @ 02:53:14]   CPU model: Intel(R) Core(TM) i5-8600K CPU @ 3.60GHz
[codecarbon INFO @ 02:53:14]   GPU count: 1
[codecarbon INFO @ 02:53:14]   GPU model: 1 x NVIDIA GeForce RTX 4070 Ti


In [7]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Evaluate on validation set


  0%|          | 0/3375 [00:00<?, ?it/s]

{'loss': 0.7713, 'grad_norm': 4.312137603759766, 'learning_rate': 1.4792899408284024e-06, 'epoch': 0.01}


[codecarbon INFO @ 02:53:33] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 02:53:33] Energy consumed for all GPUs : 0.000833 kWh. Total GPU Power : 199.82500000000002 W
[codecarbon INFO @ 02:53:33] Energy consumed for all CPUs : 0.000199 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 02:53:33] 0.001057 kWh of electricity used since the beginning.


{'loss': 0.7396, 'grad_norm': 4.900791168212891, 'learning_rate': 2.9585798816568047e-06, 'epoch': 0.02}
{'loss': 0.7045, 'grad_norm': 15.371414184570312, 'learning_rate': 4.437869822485207e-06, 'epoch': 0.03}


[codecarbon INFO @ 02:53:48] Energy consumed for RAM : 0.000050 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 02:53:48] Energy consumed for all GPUs : 0.001660 kWh. Total GPU Power : 198.96200000000002 W
[codecarbon INFO @ 02:53:48] Energy consumed for all CPUs : 0.000396 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 02:53:48] 0.002105 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 03:49:24] Energy consumed for RAM : 0.005552 kWh. RAM Power : 5.9499335289001465 W
[codecarbon INFO @ 03:49:24] Energy consumed for all GPUs : 0.189944 kWh. Total GPU Power : 54.421 W
[codecarbon INFO @ 03:49:24] Energy consumed for all CPUs : 0.044404 kWh. Total CPU Power : 47.5 W
[codecarbon INFO @ 03:49:24] 0.239900 kWh of electricity used since the beginning.


{'train_runtime': 3365.7127, 'train_samples_per_second': 80.221, 'train_steps_per_second': 1.003, 'train_loss': 0.1813771443190398, 'epoch': 3.0}


C:\Users\bes-s\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\codecarbon\output.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])


***** train metrics *****
  epoch                    =           3.0
  total_flos               = 19700545835GF
  train_loss               =        0.1814
  train_runtime            =    0:56:05.71
  train_samples_per_second =        80.221
  train_steps_per_second   =         1.003


KeyError: 'validation'

In [8]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/500 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.9533
  eval_loss               =      0.139
  eval_runtime            = 0:00:59.53
  eval_samples_per_second =    167.956
  eval_steps_per_second   =      8.398
